In [1]:
using JuAFEM
using Tensors
using TimerOutputs
using UnicodePlots
const to = TimerOutput();

INFO: Recompiling stale cache file /home/kristoffer/.julia/lib/v0.5/JuAFEM.ji for module JuAFEM.


In [2]:
import Base.flatten

In [190]:
const dim = 3
corner1 = Vec{dim}((0.0, 0.0, 0.0))
corner2 = Vec{dim}((10.0, 1.0, 1.0))
grid = generate_grid(Tetrahedron, (120, 12, 12), corner1, corner2);
# Extract the left boundary
addnodeset!(grid, "clamped", x -> norm(x[1]) ≈ 0.0);

In [191]:
# Interpolations and values
interpolation_space = Lagrange{dim, RefTetrahedron, 1}()
quadrature_rule = QuadratureRule{dim, RefTetrahedron}(1)
cellvalues = CellVectorValues(quadrature_rule, interpolation_space);
facevalues = FaceVectorValues(QuadratureRule{dim-1, RefTetrahedron}(1), interpolation_space);

In [215]:
# DofHandler
dh = DofHandler(grid)
push!(dh, :u, dim) # Add a displacement field
close!(dh)

DofHandler
  Fields:
    u dim: 3
  Total dofs: 61347
  Dofs per cell: 12

In [193]:
@time K = create_sparsity_pattern(dh);
fill!(K.nzval, 1.0);
spy(K)

  1.163711 seconds (30 allocations: 523.239 MB, 12.85% gc time)


                      Sparsity Pattern
         ┌──────────────────────────────────────────┐    
       1 │⠻⣦⡀⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
         │⣤⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
         │⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠈⠻⣦⡈⠻⣦⡈⠹⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠈⠻⣆⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠛⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⠈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡀⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣤⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [194]:
# Boundaryconditions
dbc = DirichletBoundaryConditions(dh)
# Add a homogenoush boundary condition on the "clamped" edge
add!(dbc, :u, getnodeset(grid, "clamped"), (x,t) -> [0.0, 0.0, 0.0], collect(1:dim))
close!(dbc)
t = 0.0
update!(dbc, t)

In [195]:
# Create the stiffness tensor
E = 200e9
ν = 0.3
λ = E*ν / ((1+ν) * (1 - 2ν))
μ = E / (2(1+ν))
δ(i,j) = i == j ? 1.0 : 0.0
g(i,j,k,l) = λ*δ(i,j)*δ(k,l) + μ*(δ(i,k)*δ(j,l) + δ(i,l)*δ(j,k))
C = SymmetricTensor{4, dim}(g);

In [209]:
function doassemble{dim}(cellvalues::CellVectorValues{dim}, facevalues::FaceVectorValues{dim}, 
                         K::SparseMatrixCSC, grid::Grid, dh::DofHandler, C::SymmetricTensor{4, dim})
    assem_strat = 1
    n_basefuncs = getnbasefunctions(cellvalues)
    global_dofs = zeros(Int, ndofs_per_cell(dh))

    fill!(K.nzval, 0.0)
    f = zeros(ndofs(dh))
    assembler = start_assemble(K, f)

    fe = zeros(n_basefuncs) # Local force vector
    Ke = zeros(n_basefuncs, n_basefuncs) # Local stiffness mastrix
    
    t = Vec{3}((0.0, 1e8, 0.0)) # Traction vector
    b = Vec{3}((0.0, 0.0, 0.0)) # Body force

    ɛ = [zero(SymmetricTensor{2, dim}) for i in 1:n_basefuncs]
    @inbounds for (cellcount, cell) in enumerate(CellIterator(dh))
        @timeit to "assem" begin
        fill!(Ke, 0)
        fill!(fe, 0)
        
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            for i in 1:n_basefuncs
                ɛ[i] = symmetric(shape_gradient(cellvalues, q_point, i)) 
            end
            dΩ = getdetJdV(cellvalues, q_point)
            for i in 1:n_basefuncs
                δu = shape_value(cellvalues, q_point, i)
                fe[i] += (δu ⋅ b) * dΩ
                ɛC = ɛ[i] ⊡ C
                for j in 1:n_basefuncs
                    Ke[i, j] += (ɛC ⊡ ɛ[j]) * dΩ
                end
            end
        end
        
        for face in 1:nfaces(cell)
            if onboundary(cell, face) && (cellcount, face) ∈ getfaceset(grid, "right")
                reinit!(facevalues, cell, face)
                for q_point in 1:getnquadpoints(facevalues)
                    dΓ = getdetJdV(facevalues, q_point)
                    for i in 1:n_basefuncs
                        δu = shape_value(facevalues, q_point, i)
                        fe[i] += (δu ⋅ t) * dΓ
                    end
                end
            end
        end
        celldofs!(global_dofs, cell)
        assemble!(assembler, fe, Ke, global_dofs)
        end # timer
    end
    #K = end_assemble(assembler)
    return K, f
end

doassemble (generic function with 1 method)

In [214]:
reset_timer!(to)
K, f = doassemble(cellvalues, facevalues, K, grid, dh, C);
print_timer(to; linechars = :ascii)

 ------------------------------------------------------------------
                           Time                   Allocations      
                   ----------------------   -----------------------
 Tot / % measured:      252ms / 92.6%           32.7MiB / 86.5%    

 Section   ncalls     time   %tot     avg     alloc   %tot      avg
 ------------------------------------------------------------------
 assem      86.4k    233ms   100%  2.70μs   28.3MiB  100%      344B
 ------------------------------------------------------------------

In [183]:
# Account for boundary conditions and solve
u = zeros(ndofs(dh))
d_free = JuAFEM.free_dofs(dbc)
d_pres = dbc.dofs
@time u_free = cholfact(Symmetric(K[d_free, d_free])) \ (f[d_free] - K[d_free, d_pres] * dbc.values)
u[d_free] = u_free 
apply!(u, dbc);

  0.168582 seconds (85 allocations: 50.701 MB, 62.79% gc time)


In [184]:
# Save file
vtkfile = vtk_grid("cantilever", dh, u)
vtk_save(vtkfile)

1-element Array{String,1}:
 "cantilever.vtu"

In [185]:
Base.Test.@test maximum(u)  ≈ 1.919600482922295
println("Cantilever successful")

Cantilever successful
